##Satellite Image Classification with EuroSAT & ResNet18

In this project I demonstrate how to build, fine tune, and deploy a satellite image classification model using transfer learning with a pretrained ResNet18 Model.






ResNet18 is a convolutional neural network designed to learn patterns in images.The model is trained on the EuroSETdataset, which contains over 27,000 Sentinel-2 satellite images labeled into 10 land use and land cover categories.
By fine-tuning the final layer of the model, I achieved over 93% accuracy on both the training and validation datasets resulting in an efficient model that can be used for satellite imagery classification.





**Installing & Importing Necessary Packages**

In [1]:
#Installing core Pytorch libraries for deep learning
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118

#Installing for image handling
!pip install opencv-python-headless

#Installing for displaying results
!pip install tqdm matplotlib

#Installing to load dataset
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [2]:
import torch
import matplotlib, matplotlib.pyplot as plt
from torch import optim
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm


**Loading Data**


The EuroSAT dataset consists of satellite images taken by the Sentinel-2 satellite mission and is used to classify different types of land use and land cover. Each image is labeled with one of 10 categories that describe the primary type of terrain or human activity visible in the image. These include natural landscapes (like forests and rivers) as well as human-developed areas (such as industrial zones or residential neighborhoods).




In [ ]:
#Loding EuroSAT dataset from Hugging Face
dataset = load_dataset("blanchon/EuroSAT_RGB")

#Storing class names
label_names = dataset["train"].features["label"].names
print("Class labels:", label_names)

#Viewing Dataset (one image per class)
import matplotlib.pyplot as plt

seen_classes = set()
images_per_class = {}

# Looping through dataset and collecting one image per class
for item in dataset["train"]:
    label = item["label"]
    if label not in seen_classes:
        images_per_class[label] = item["image"]
        seen_classes.add(label)
    if len(seen_classes) == len(label_names):
        break

#Displaying one image per class
plt.figure(figsize=(15, 6))
for i, label in enumerate(sorted(images_per_class.keys())):
    plt.subplot(2, 5, i + 1)
    plt.imshow(images_per_class[label])
    plt.title(f"Class: {label_names[label]}")
    plt.axis("off")

plt.suptitle("EuroSAT Data Preview")
plt.tight_layout()
plt.show()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Preprocessing Data for Input into ResNET Model**

I converted the data into Python lists of images and labels to ensure compatibility with Pytorch. Next I split the dataset into a train and validation sets. After creating test and validation sets, I applied the neccessary transformations needed for input in the ResNET model including resizing images and converting them to tensors.

In [ ]:

raw_dataset = load_dataset("blanchon/EuroSAT_RGB")["train"]

# Creating lists to store individual image-label pairs
images = []
labels = []

# Extracting each image-label pair in the dataset with a for loop
for ex in raw_dataset:
    img = ex["image"]        # Getting the image (PIL.Image)
    label = ex["label"]      # Getting the corresponding class label (0–9)

    #Unwrapping if image is stored as a single-item list [PIL.Image]
    if isinstance(img, list):
        img = img[0]

    images.append(img)
    labels.append(label)

In [ ]:
#Splitting the data into training and validation sets
# Using stratified sampling to maintain class balance in both set

train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    images,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)


In [ ]:
#Transforming images to be compatible with ResNet
# Resize the image to 224x224, convert to tensor, normalize based on ImageNet

#Resizing images to 224X224, converting to tensors, normalizing based on ImageNet
resnet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
#Creating a PyTorch Dataset class from our image and class list
class EuroSATDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)  # Converting label to tensor (required for CrossEntropyLoss)

        if self.transform:
            img = self.transform(img)

        # Returning a dictionary with transformed image and label
        return {"image": img, "label": label}

In [ ]:
#Wrapping the training data in a custom PyTorch Dataset
train_dataset = EuroSATDataset(train_imgs, train_labels, transform=resnet_transform)
val_dataset   = EuroSATDataset(val_imgs, val_labels, transform=resnet_transform)

#Creating DataLoaders to efficiently load data in batches
train_loader = DataLoader(
    train_dataset,
    batch_size=32,    # Number of samples per training batch
    shuffle=True      # Shuffling training data for better generalization
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32
)


In [ ]:
# Checking one batch to ensure imaes are correctly batched and sized and the labels are the correct type (torch.long)
batch = next(iter(train_loader))

images = batch["image"]
labels = batch["label"]

print("Image batch shape:", images.shape)     # [32, 3, 224, 224]
print("Label batch dtype:", labels.dtype)     # torch.int64 (aka torch.long)
print("Label range:", labels.min().item(), "to", labels.max().item())  #0 to 9


**Loading the ResNET18 Model**

Next I loaded our base ResNET18 model. I froze all early layers and replaced the final classification layer to output the 10 EuroSAT classifications.

In [ ]:
#Loading the ResNet18 model and pretrained ImageNet weights
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

#Freezing early layers of convolutional base
for param in model.parameters():
    param.requires_grad = False

#Replacing the final fully connected layer
model.fc = nn.Linear(model.fc.in_features, 10)  # 10 EuroSAT land cover classes


In [ ]:
#Setting model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Defining the optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

**Training ResNet18 using Transfer Learning**

I trained the model for 10 epochs using the Adam optimizer and cross-entropy loss. Training and validation accuracy are printed per epoch. The model quickly reaches high accuracy due to transfer learning from ImageNet.


In [ ]:
#Training the model for a given number of epochs using the training DataLoader
def train_model(model, train_loader, criterion, optimizer, device, epochs=10):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0   # Accumulates loss for reporting
        correct = 0          # Counter for correct predictions
        total = 0            # Total number of samples processed

        # Adding progress bar to show training progress
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        # Iterating over each batch of training data
        for batch in loop:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            #Updating progress bar with current loss and accuracy
            loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

        # Printing epoch summary
        print(f"✅ Epoch {epoch+1} complete | Accuracy: {100 * correct / total:.2f}% | Loss: {running_loss:.4f}")


In [ ]:
#training model
train_model(model, train_loader, criterion, optimizer, device, epochs=10)


**Validating Model on Unseen Data**

Next, I computd the final validation accuracy after training. The model achieved 93% accuracy confirming strong generalization on unseen satellite imagery.


In [ ]:
# Evaluating the model on the validation dataset
def evaluate_model(model, val_loader, criterion, device, return_acc=False):
    model.eval()

#count of current predictions, total number of samples, accumulated loss
    correct = 0
    total = 0
    total_loss = 0

    # Inhibiting gradient tracking to save memory and compute
    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    # Calculating accuracy as a percentage
    accuracy = 100 * correct / total

    # 📊 Print the overall validation metrics
    print(f"📊 Validation Accuracy: {accuracy:.2f}% | Loss: {total_loss:.4f}")




In [ ]:
#validating model
val_acc = evaluate_model(model, val_loader, criterion, device, return_acc=True)


**Saving The Model**

I saved the trained model as a `.pth` file and loaded it into a FastAPI deployment environment that accepts satellit images, returns predicted land use labels, and generates a downloadable excel file with thumbnails, filesnames,and classifications.




In [ ]:
torch.save(model.state_dict(), "eurosat_resnet18_finetuned.pth")
print("Model saved!")

from google.colab import files
files.download("eurosat_resnet18_finetuned.pth")


##Conclusion

This project demonstrates how transfer learning and geospatial data can be combined to automate land use classification from satellite images.
This project could be extended to include more classes, real-time satellite data, or further model tuning.
